In [16]:
import folium
from folium.features import DivIcon
import folium.plugins
import numpy as np
import random

def show_shops_map(shops, my_map=None, day=0):
    def draw_shop(shop, color, fill_opacity=0.6, radius=2, txt=''):
        lat, lon = shop["location"]
        return folium.CircleMarker(
            location=[lat, lon], 
            popup = f'name:{shop["shopname"]}\n'+txt, 
            radius = max(0.01, radius), 
            color = color, 
            fill_opacity = fill_opacity,
            weight = 1, # gruix perimetre
            fill = True,
            fill_color = color
        )

    mymap = folium.Map(location=(41.401361, 2.174317), zoom_start=13, tiles="cartodbpositron", control_scale=True)

    all_shops_group = folium.FeatureGroup('Shops', overlay=True) # Pots fer capes per filtrar a posteriori
    for _id, shop in shops.items():
        draw_shop(
            shop,
            color = 'green' if shop["n_trans"] > 0 else 'red',
            fill_opacity = 0.1 if shop["n_trans"] > 0 else 0.3,
            radius=5, #(shop["n_trans"]**0.5716)*1.2, #formula per matenir proporcions visuals humanes
            txt=f'Transctions:{np.round(shop["n_trans"]**0.5716*1.2, 2)}',
        ).add_to(all_shops_group)

    all_shops_group.add_to(mymap)
    folium.LayerControl(collapsed=False).add_to(mymap)
    mymap.keep_in_front(all_shops_group)
    mymap.add_child(folium.plugins.Fullscreen())
    
    title_html = '''
             <h3 align="center" style="font-size:16px"><b>day: {}</b></h3>
             '''.format(day)
    mymap.get_root().html.add_child(folium.Element(title_html))
    return mymap

In [2]:
import pandas as pd

def preprocessDF(path):
    shops = pd.read_csv(path, index_col=0)
    shops["n_trans"] = 0
    shops.n_trans = shops.n_trans.map(lambda x: random.randint(0,30))
    shops = shops.set_index(shops._id)
    shops.location = shops.location.map(lambda x: [y.strip("[ '']") for y in x.split(",")])
    shops.tags = shops.tags.map(lambda x: [y.strip("[ '']") for y in x.split(",")])
    shops_dict = shops.to_dict(orient='index')
    return shops_dict

In [3]:
import folium
from selenium import webdriver
import os
import time

def make_screenshot(m, img_name):
    m.save("index.html")
    mapUrl = 'file://{0}/{1}'.format(os.getcwd(), "index.html")
    driver = driver=webdriver.Firefox(executable_path="./geckodriver",log_path=None)
    driver.get(mapUrl)
    time.sleep(5)
    driver.save_screenshot(f'./img/{img_name}.png')
    driver.quit()

In [17]:
path = "/Users/tomas.gadea/tomasgadea/ACADEMIC/GCED/q6/PE/beco-project/backend/data_base/map/shops.csv"
shops_dict = preprocessDF(path)
m = show_shops_map(shops_dict, day=10)
#make_screenshot(m, f"{i:03d}")
m

In [8]:
import folium

loc = 'Corpus Christi'
title_html = '''
             <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(loc)   

m = folium.Map(location=[27.783889, -97.510556],
               zoom_start=12)

m.get_root().html.add_child(folium.Element(title_html))

m.save('map-with-title.html')
m